In [ ]:
import sys
sys.path.append('./scripts/src/')

import numpy as np
import os

from utils import show
import utils as ut
import image_processes as imgp
import map_processes as mapp
import cv2
import os
import numpy as np
from skimage.util import invert
from skimage.io import imread
import post_processes as pp

%load_ext autoreload
%autoreload 2

In [35]:
input = "border1.png"
project_name = os.path.basename(input).split('.')[0]
input_path = 'scripts/inputs'
output_path = 'scripts/outputs/' + project_name
current_dir = os.getcwd()
img_path = os.path.join(current_dir, input_path, input)

if not os.path.exists(output_path):
    os.makedirs(output_path)

def get_path(filename):
    return os.path.join(current_dir, output_path, filename)


In [36]:
# as_gray to be able to get height and width with shape
image = imread(img_path, as_gray=True)
height, width = image.shape
 
img_to_display = (image * 255).astype(np.uint8)
# show(img_to_display)

In [ ]:
quantization_config = mapp.QuantizationConfig(5, False)
quantization_config.show()

In [ ]:
# Convert artist skeleton to bool image
skeleton = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
print("Skeleton shape:", skeleton.shape)

has_alpha = skeleton.shape[2] == 4

# print shape
print(skeleton.shape)

if has_alpha:
    mask = skeleton[:, :, 3] == 0
    skeleton[mask, 0:3] = [255, 255, 255]
    skeleton[~mask, 0:3] = [0, 0, 0]
else :
    mask = np.all(skeleton == [255, 255, 255], axis=-1)
    skeleton = np.zeros_like(skeleton)
    skeleton[mask] = [255, 255, 255]

skeleton = imgp.convert_image_to_grayscale(skeleton, False)
skeleton = skeleton.astype(bool)

if image.dtype != np.uint8:
    image = (image * 255).astype(np.uint8)

show(skeleton)


In [ ]:
# CREATE THICK BORDER MAP FROM ARTIST SKELETON FOR CONTOUR DETECTION
invertedSkeleton = invert(skeleton)
color = imgp.hex_to_rgba("#000000")
border_map = imgp.thickening(invertedSkeleton, 2, 1, 80, color, False, False)

# Convert to usable format
mask = border_map[:, :, 3] == 0
border_map[mask, 0:3] = [255, 255, 255]
border_map = cv2.cvtColor(border_map, cv2.COLOR_BGRA2BGR)

border_map = imgp.convert_image_to_grayscale(border_map, True)

# Save
ut.save_cv_image(output_path, project_name + "_debug_thick_skeleton.png", border_map)

In [ ]:
# Estimate region count / get contours
# With post process
(
    contours,
    points,
    shape_informations,
    duplicated_contours,
) = imgp.extract_refined_contour_info(
    border_map,
    2,
    cv2.RETR_TREE,
    verbose=False,
    treshold_contour_distance=10,
    min_contour_area=50,
    enable_duplicate_island_provinces_post_process=True,
    enable_small_provinces_post_process=True,
)


# Debug duplicate islands
duplicated_island_debug_map = mapp.debug_duplicated_islands(
    skeleton, duplicated_contours, False
)
cv2.imwrite(get_path(project_name+"_debug_duplicated_island_map.png"),
            duplicated_island_debug_map)

In [41]:
# Calculate shape informations
enclosed_circles_img, ellipses_img, centers_img = mapp.generate_debug_shape_images(
    shape_informations, skeleton
)

# show(enclosed_circles_img)
# show(ellipses_img)
# show(centers_img)

ut.save_cv_image(output_path, project_name+"_debug_enclosed_circles.png", enclosed_circles_img)
ut.save_cv_image(output_path, project_name+"_debug_ellipses.png", ellipses_img)
ut.save_cv_image(output_path, project_name+"_debug_centers.png", centers_img)

In [42]:
# Find adjacent provinces
adjacency_list = mapp.find_adjacent_provinces(contours)

In [ ]:
# Draw contours
filled_image = imgp.draw_contours(skeleton, contours, False)
# Test filling
filled_image2 = imgp.fill_contours(skeleton, filled_image, points, True, shape_informations, True)
# Check for holls in the script generated skeleton :
#   -   if two neighbors have the same color, then there is a holl
#   -   if the color of a same province is different in the two images -> may be a problem -> investigate
ut.save_cv_image(output_path, project_name+'_debug_holl_check_map_contour.png', filled_image)
ut.save_cv_image(output_path, project_name+'_debug_holl_check_map.png', filled_image2)

In [ ]:
# Draw bitmap and create province data views
province_data_views, map_colors, map_debug = mapp.do_map_quantization(
    skeleton, points, shape_informations, False, True, quantization_config=quantization_config, font_size=0.27)

cv2.imwrite(get_path(project_name+"_bitmap_whitelines.png"),
            cv2.cvtColor(map_colors, cv2.COLOR_BGR2RGB))

if map_debug is not None:
    cv2.imwrite(get_path(project_name+"_bitmap_debug.png"),
                cv2.cvtColor(map_debug, cv2.COLOR_BGR2RGB))

In [45]:
# Create Provinces Json
provinces = mapp.create_province_json(province_data_views)
ut.write_json(provinces, output_path, project_name + "_provinces_raw")

In [ ]:
# Remove white lines
# TODO : improve handling of black points (can currently induce irregularities in the lines)
# Be sure that there is no black background in the bitmap input
bitmap_raw = mapp.remove_white_lines_from_color_map(map_colors)

show(bitmap_raw)
cv2.imwrite(
    get_path(project_name+"_bitmap_raw.png"),
    cv2.cvtColor(bitmap_raw, cv2.COLOR_BGR2RGB),
)

In [ ]:
# Detect fake provinces
fake_provinces_ids = pp.find_fake_provinces(bitmap_raw, province_data_views, quantization_config).tolist()
print(fake_provinces_ids)
ut.write_json(fake_provinces_ids, output_path, "fake_provinces", False)

In [48]:
# Get neighbors from fake provinces (are likely to be real litoral provinces)
fake_province_neighbors = []

fake_provinces = [
    province for province in province_data_views if province.id in fake_provinces_ids
]

for province in fake_provinces:
    fake_province_neighbors.extend(province.landNeighbours)


In [ ]:
# Clean data: remove fake provinces
province_data_pp1 = pp.remove_ids_from_provinces_list(fake_provinces_ids, province_data_views, True, True)
print(len(province_data_views), len(province_data_pp1))

In [ ]:
# CREATE BORDER MAP AS A GAME ASSET
invertedSkeleton = invert(skeleton)
color = imgp.hex_to_rgba("#151515")
asset_border_map = imgp.thickening(invertedSkeleton, 2, 3, 80, color, True, False)
# Add skeleton on top of asset_border_map
# TODO
ut.save_cv_image(output_path, project_name + "_map_borders_generated.png", asset_border_map)

In [51]:
# Generate debug map
mask = imgp.create_blank_image(width, height, (60, 20, 90, 255))
ut.save_cv_image(output_path, project_name + "_map_debug.png", mask)